In [1]:
using JuMP

In [2]:
using CPLEX

[ Info: Precompiling CPLEX [a076750e-1247-5638-91d2-ce28b192dca0]
ERROR: LoadError: CPLEX not properly installed. Please run Pkg.build("CPLEX")
Stacktrace:
 [1] error(s::String)
   @ Base .\error.jl:35
 [2] top-level scope
   @ C:\Users\user\.julia\packages\CPLEX\rmvSt\src\CPLEX.jl:7
 [3] include
   @ .\Base.jl:419 [inlined]
 [4] include_package_for_output(pkg::Base.PkgId, input::String, depot_path::Vector{String}, dl_load_path::Vector{String}, load_path::Vector{String}, concrete_deps::Vector{Pair{Base.PkgId, UInt64}}, source::Nothing)
   @ Base .\loading.jl:1554
 [5] top-level scope
   @ stdin:1
in expression starting at C:\Users\user\.julia\packages\CPLEX\rmvSt\src\CPLEX.jl:1
in expression starting at stdin:1


LoadError: Failed to precompile CPLEX [a076750e-1247-5638-91d2-ce28b192dca0] to C:\Users\user\.julia\compiled\v1.8\CPLEX\jl_220E.tmp.

In [6]:
using HiGHS
using Plots; plotly()

Plots.PlotlyBackend()

In [7]:
using VegaLite  # to make some nice plots
using DataFrames, CSV, PrettyTables
using FileIO

In [8]:
ENV["COLUMNS"]=120; # Set so all columns of DataFrames and Matrices are displayed

datadir = joinpath("kr_data")

"kr_data"

In [9]:
gen_info = CSV.read("kGenerators_data.csv", DataFrame);
fuels = CSV.read("Fuels_data.csv", DataFrame);

In [56]:
loads = CSV.read("Demand_20220701_2.csv", DataFrame);
gen_variable = CSV.read("Generators_variability.csv", DataFrame);

In [57]:
for f in [gen_info, fuels, loads, gen_variable]
    rename!(f,lowercase.(names(f)))
end

In [58]:

select!(gen_info, 1:26, :stor)
gen_df = outerjoin(gen_info,  fuels, on = :fuel) # load in fuel costs and add to data frame
rename!(gen_df, :cost_per_mmbtu => :fuel_cost)
gen_df[ismissing.(gen_df[:,:fuel_cost]), :fuel_cost] .= 0
gen_df[!, :is_variable] .= false 
gen_df[in(["onshore_wind_turbine","small_hydroelectric","solar_photovoltaic"]).(gen_df.resource), :is_variable] .= true;


In [59]:
gen_df.gen_full = lowercase.(gen_df.region .* "_" .* gen_df.resource .* "_" .* string.(gen_df.cluster) .* ".0");
gen_df = gen_df[gen_df.existing_cap_mw .> 0,:];
gen_variable.hour = mod.(gen_variable.hour .- 9, 8760) .+1 
sort!(gen_variable, :hour)

Row,hour,wec_sdge_biomass_1.0,wec_sdge_hydroelectric_pumped_storage_1.0,wec_sdge_natural_gas_fired_combined_cycle_1.0,wec_sdge_natural_gas_fired_combined_cycle_2.0,wec_sdge_natural_gas_fired_combined_cycle_3.0,wec_sdge_natural_gas_fired_combined_cycle_4.0,wec_sdge_natural_gas_fired_combustion_turbine_1.0,wec_sdge_natural_gas_fired_combustion_turbine_2.0,wec_sdge_natural_gas_fired_combustion_turbine_3.0,wec_sdge_natural_gas_fired_combustion_turbine_4.0,wec_sdge_natural_gas_fired_combustion_turbine_5.0,wec_sdge_natural_gas_fired_combustion_turbine_6.0,wec_sdge_natural_gas_fired_combustion_turbine_7.0,wec_sdge_natural_gas_fired_combustion_turbine_8.0,wec_sdge_natural_gas_fired_combustion_turbine_9.0,wec_sdge_natural_gas_fired_combustion_turbine_10.0,wec_sdge_natural_gas_fired_combustion_turbine_11.0,wec_sdge_natural_gas_fired_combustion_turbine_12.0,wec_sdge_onshore_wind_turbine_1.0,wec_sdge_small_hydroelectric_1.0,wec_sdge_solar_photovoltaic_1.0,wec_sdge_naturalgas_ccccsavgcf_0.0,wec_sdge_naturalgas_ccavgcf_0.0,wec_sdge_naturalgas_ctavgcf_0.0,wec_sdge_battery_0.0,wec_sdge_naturalgas_ccs100_0.0,wec_sdge_landbasedwind_ltrg1_1.0,wec_sdge_landbasedwind_ltrg1_2.0,wec_sdge_utilitypv_losangeles_1.0,wec_sdge_utilitypv_losangeles_2.0
,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Float64,Float64,Float64,Int64,Int64,Int64,Int64,Int64,Float64,Float64,Float64,Float64
1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0.1694,0.299,0.0,1,1,1,1,1,0.131448,0.235186,0.0,0.0
2,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0.1658,0.2987,0.0,1,1,1,1,1,0.162792,0.23255,0.0,0.0
3,3,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0.1502,0.2984,0.0,1,1,1,1,1,0.169585,0.197422,0.0,0.0
4,4,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0.1455,0.2981,0.0,1,1,1,1,1,0.115781,0.168985,0.0,0.0
5,5,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0.0932,0.2978,0.0,1,1,1,1,1,0.0635784,0.0962406,0.0,0.0
6,6,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0.0717,0.2974,0.0,1,1,1,1,1,0.0736582,0.0736047,0.0,0.0
7,7,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0.0211,0.2971,0.0,1,1,1,1,1,0.191722,0.0231735,0.0,0.0
8,8,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0.0216,0.2968,0.0488,1,1,1,1,1,0.141564,0.022771,0.160355,0.126487
9,9,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0.0216,0.2965,0.3515,1,1,1,1,1,0.172937,0.0334803,0.530314,0.517171


In [60]:
loads.hour = mod.(loads.hour .- 9, 8760) .+ 1
sort!(loads, :hour);
gen_variable     
describe(gen_variable)


Row,variable,mean,min,median,max,nmissing,eltype
,Symbol,Float64,Real,Float64,Real,Int64,DataType
1,hour,4380.5,1,4380.5,8760,0,Int64
2,wec_sdge_biomass_1.0,1.0,1,1.0,1,0,Int64
3,wec_sdge_hydroelectric_pumped_storage_1.0,1.0,1,1.0,1,0,Int64
4,wec_sdge_natural_gas_fired_combined_cycle_1.0,1.0,1,1.0,1,0,Int64
5,wec_sdge_natural_gas_fired_combined_cycle_2.0,1.0,1,1.0,1,0,Int64
6,wec_sdge_natural_gas_fired_combined_cycle_3.0,1.0,1,1.0,1,0,Int64
7,wec_sdge_natural_gas_fired_combined_cycle_4.0,1.0,1,1.0,1,0,Int64
8,wec_sdge_natural_gas_fired_combustion_turbine_1.0,1.0,1,1.0,1,0,Int64
9,wec_sdge_natural_gas_fired_combustion_turbine_2.0,1.0,1,1.0,1,0,Int64


In [61]:
gen_variable_long = stack(gen_variable, 
                        Not(:hour), 
                        variable_name=:gen_full,
                        value_name=:cf);


In [62]:
function value_to_df_puvw(var1, var2, var3, var4)
    solution1 = DataFrame(var1.data, :auto)
    ax1 = var1.axes[1]
    ax2 = var1.axes[2]
    cols = names(solution1)
    insertcols!(solution1, 1, :r_id => ax1)
    solution1 = stack(solution1, Not(:r_id), variable_name=:hour)
    solution1.hour = foldl(replace, [cols[i] => ax2[i] for i in 1:length(ax2)], init=solution1.hour)
    rename!(solution1, :value => :gen)
    solution1.hour = convert.(Int64,solution1.hour)
    solution2 = DataFrame(var2.data, :auto)
    ax1 = var2.axes[1]
    ax2 = var2.axes[2]
    cols = names(solution2)
    insertcols!(solution2, 1, :r_id => ax1)
    solution2 = stack(solution2, Not(:r_id), variable_name=:hour)
    solution2.hour = foldl(replace, [cols[i] => ax2[i] for i in 1:length(ax2)], init=solution2.hour)
    rename!(solution2, :value => :u_binary)
    solution2.hour = convert.(Int64,solution2.hour)
    solution3 = DataFrame(var3.data, :auto)
    ax1 = var3.axes[1]
    ax2 = var3.axes[2]
    cols = names(solution3)
    insertcols!(solution3, 1, :r_id => ax1)
    solution3 = stack(solution3, Not(:r_id), variable_name=:hour)
    solution3.hour = foldl(replace, [cols[i] => ax2[i] for i in 1:length(ax2)], init=solution3.hour)
    rename!(solution3, :value => :v_binary)
    solution3.hour = convert.(Int64,solution3.hour)
    solution4 = DataFrame(var4.data, :auto)
    ax1 = var4.axes[1]
    ax2 = var4.axes[2]
    cols = names(solution4)
    insertcols!(solution4, 1, :r_id => ax1)
    solution4 = stack(solution4, Not(:r_id), variable_name=:hour)
    solution4.hour = foldl(replace, [cols[i] => ax2[i] for i in 1:length(ax2)], init=solution4.hour)
    rename!(solution4, :value => :w_binary)
    solution4.hour = convert.(Int64,solution4.hour)
    return solution1, solution2, solution3, solution4
end




value_to_df_puvw (generic function with 1 method)

In [67]:

function unit_commitment_multi_time(gen_df, loads_multi, gen_variable_multi)
UC = Model(QPDAS)

# Define sets based on data
# Thermal resources for which unit commitment constraints apply
G_thermal = gen_df[gen_df[!,:up_time] .> 0,:r_id] 
# Non-thermal resources for which unit commitment constraints do NOT apply 
G_nonthermal = gen_df[gen_df[!,:up_time] .== 0,:r_id]
G_var = gen_df[gen_df[!,:is_variable] .== 1,:r_id]
G_nonvar = gen_df[gen_df[!,:is_variable] .== 0,:r_id]
# Non-variable and non-thermal resources
G_nt_nonvar = intersect(G_nonvar, G_nonthermal)
G = gen_df.r_id
T = loads_multi.hour
T_red = loads_multi.hour[1:end-1]  # time periods used for ramp constraints

# Generator capacity factor time series for variable generators
gen_var_cf = innerjoin(gen_variable_multi, 
                gen_df[gen_df.is_variable .== 1 , 
                    [:r_id, :gen_full, :existing_cap_mw]], 
                on = :gen_full)
    
# Decision variables   
@variables(UC, begin
    p1[G, T]  >= 0  
    p[G, T]  >= 0  
    u[G_thermal, T], Bin 
    # ED를 UC로 변경하기 위해 운영여부 u[g,t]변수 추가(Binary)
    v[G_thermal, T], Bin
    # 정지계획 추가를 위해 정지(Shut-dowm)여부 w[g,t]변수 추가(Binary)
    w[G_thermal, T], Bin
    # output 여부 변수(Binary)
    o[G_thermal, T], Bin
 end)


    # Objective function
@objective(UC, Min, 
    sum((gen_df[gen_df.r_id .== i,:heat_rate_mmbtu_per_mwh][1] * gen_df[gen_df.r_id .== i,:fuel_cost][1] +
        gen_df[gen_df.r_id .== i,:var_om_cost_per_mwh][1]) * p[i,t] 
                    for i in G_nonvar for t in T) + 
    sum(gen_df[gen_df.r_id .== i,:var_om_cost_per_mwh][1] * p[i,t] 
                    for i in G_var for t in T) +
    sum(gen_df[gen_df.r_id .== i,:start_cost_per_mw][1] * gen_df[gen_df.r_id .== i,:existing_cap_mw][1] * v[i,t] 
                    for i in G_thermal for t in T) +
    sum(500*u[i,t] for i in G_thermal for t in T) #기동고정비 항목이나 현재 그냥 500 추후 수정
)



# output 변수 관련 제약
@constraint(UC, ou[i in G_thermal, t in T], 
            o[i,t] <= u[i,t])
@constraint(UC, o1[i in G_thermal, t in T], 
            p[i,t] * o[i,t] >= gen_df[gen_df.r_id .== i,:existing_cap_mw][1] *    
            gen_df[gen_df.r_id .== i,:min_power][1] * o[i,t])

@constraint(UC, o2[i in G_thermal, t in T], 
            p[i,t] * o[i,t] >= p[i,t] - gen_df[gen_df.r_id .== i,:existing_cap_mw][1] *
            gen_df[gen_df.r_id .== i,:min_power][1])
            
# u, v, w 관계 제약식
@constraint(UC, Binary_Formulations[i in G_thermal, t in T_red], 
           u[i,t+1] - u[i,t] == v[i,t+1] - w[i,t+1])

# 동일 시간에 v, w가 동시에 1 불가      
@constraint(UC, vw[i in G_thermal, t in T], 
            v[i,t] + w[i,t] <= 1)

#=
 intersection 연산이 느린 것 같아 다른 것으로 일단 대체
# Minimum Up Time Constraints
@constraint(UC, MUT[i in G_thermal, t in T],
u[i, t] >= sum(v[i, tt] 
                for tt in intersect(T,
                    (t-gen_df[gen_df.r_id .== i,:up_time][1]):t)))

# Minimum Down Time Constraints           
@constraint(UC, MDT[i in G_thermal, t in T],
1-u[i, t] >= sum(w[i, tt] 
                for tt in intersect(T,
                    (t-gen_df[gen_df.r_id .== i,:down_time][1]):t)))
=#

T_MUT(i) = T[1:end-1-gen_df[gen_df.r_id .== i,:up_time][1]]
T_MDT(i) = T[1:end-1-gen_df[gen_df.r_id .== i,:down_time][1]]

# Minimum Up Time Constraints
@constraint(UC, MUT[i in G_thermal, t in T_MUT(i)], 
            sum(u[i,t+1] for t = t:t+gen_df[gen_df.r_id .== i,:up_time][1]) >= (u[i,t+1] - u[i,t])*min(gen_df[gen_df.r_id .== i,:up_time][1],(T_period[end]-t-1)))

# Minimum Down Time Constraints           
@constraint(UC, MDT[i in G_thermal, t in T_MDT(i)], 
            sum((1-u[i,t+1]) for t = t:t+gen_df[gen_df.r_id .== i,:down_time][1]) >= (u[i,t] - u[i,t+1])*min(gen_df[gen_df.r_id .== i,:down_time][1],(T_period[end]-t-1)))

# Demand constraint
@constraint(UC, cDemand[t in T], 
    sum(p[i,t] for i in G) == loads_multi[loads_multi.hour .== t, :demand][1])

#= Capacity constraints (non-variable generation)에 u 항목을 추가
   p_min*u <= p <= p_max*u 조건인데, p_min*u <= p*u <= p_max*u 조건과 동일한 조건이라고 판단 
@constraint(UC, Cap_thermal_min[i in G_thermal, t in T], 
   p[i,t] >= u[i,t] * gen_df[gen_df.r_id .== i,:existing_cap_mw][1] *
                   gen_df[gen_df.r_id .== i,:min_power][1]) =#

@constraint(UC, Cap_thermal_max[i in G_thermal, t in T], 
   p[i,t] <= u[i,t] * gen_df[gen_df.r_id .== i,:existing_cap_mw][1])

# 2. non-variable generation not requiring commitment
@constraint(UC, Cap_nt_nonvar[i in G_nt_nonvar, t in T], 
   p[i,t] <= gen_df[gen_df.r_id .== i,:existing_cap_mw][1])

# Variable generation capacity constraints
@constraint(UC, Cap_var[i in 1:nrow(gen_var_cf)], 
        p[gen_var_cf[i,:r_id], gen_var_cf[i,:hour] ] <= 
                    gen_var_cf[i,:cf] *
                    gen_var_cf[i,:existing_cap_mw])

# Ramp up constraints
@constraint(UC, RampUp[i in G, t in T_red], 
    p[i,t+1] - p[i,t] <= gen_df[gen_df.r_id .== i,:existing_cap_mw][1] * 
                             gen_df[gen_df.r_id .== i,:ramp_up_percentage][1] )

# Ramp down constraints
@constraint(UC, RampDn[i in G, t in T_red], 
    p[i,t] - p[i,t+1] <= gen_df[gen_df.r_id .== i,:existing_cap_mw][1] * 
                             gen_df[gen_df.r_id .== i,:ramp_dn_percentage][1] )

# Solve statement (! indicates runs in place)
optimize!(UC)

# Dataframe of optimal decision variables
solution = value_to_df_puvw(value.(p), value.(u), value.(v), value.(w))
solution = outerjoin(solution[1], solution[2], solution[3],solution[4], on = [:r_id,:hour])
sort!(solution, [:r_id,:hour])
    return (
        solution = solution, 
        cost = objective_value(UC),
    )
end


unit_commitment_multi_time (generic function with 1 method)

In [68]:

n=260
T_period = 1:15
loads_multi = loads[in.(loads.hour,Ref(T_period)),:]
gen_variable_multi = gen_variable_long[in.(gen_variable_long.hour,Ref(T_period)),:]


Row,hour,gen_full,cf
,Int64,String,Float64
1,1,wec_sdge_biomass_1.0,1.0
2,2,wec_sdge_biomass_1.0,1.0
3,3,wec_sdge_biomass_1.0,1.0
4,4,wec_sdge_biomass_1.0,1.0
5,5,wec_sdge_biomass_1.0,1.0
6,6,wec_sdge_biomass_1.0,1.0
7,7,wec_sdge_biomass_1.0,1.0
8,8,wec_sdge_biomass_1.0,1.0
9,9,wec_sdge_biomass_1.0,1.0


In [69]:
gen_df_sens = copy(gen_df)
gen_df_sens[gen_df_sens.resource .== "solar_photovoltaic",
    :existing_cap_mw] .= 3500

gen_df

Row,r_id,resource,region,existing_cap_mw,num_units,cap_size,var_om_cost_per_mwh,var_om_cost_per_mwh_in,fuel,heat_rate_mmbtu_per_mwh,heat_rate_mmbtu_mwh_iqr,heat_rate_mmbtu_mwh_std,min_power,ramp_up_percentage,ramp_dn_percentage,start_cost_per_mw,start_fuel_mmbtu_per_mw,up_time,down_time,self_disch,eff_up,eff_down,ratio_power_to_energy,min_duration,max_duration,cluster,stor,fuel_cost,co2_content_tons_per_mmbtu,is_variable,gen_full
,Int64?,String?,String15?,Float64?,Int64?,Float64?,Float64?,Int64?,String31,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Int64?,Float64?,Int64?,Int64?,Int64?,Float64?,Float64?,Float64?,Int64?,Int64?,Int64?,Int64?,Float64?,Float64?,Bool,String
1,1,biomass,WEC_SDGE,21.4,20,1.07,5.234,0,None,12.76,1.471,4.147,0.71,1.0,1.0,0,0.0,0,0,0,1.0,1.0,1.0,0,0,1,0,0.0,0.0,false,wec_sdge_biomass_1.0
2,2,hydroelectric_pumped_storage,WEC_SDGE,42.0,2,21.0,0.0,0,None,0.0,0.0,0.0,0.0,1.0,1.0,0,0.0,0,0,0,0.866,0.866,0.0063,0,0,1,1,0.0,0.0,false,wec_sdge_hydroelectric_pumped_storage_1.0
3,3,natural_gas_fired_combined_cycle,WEC_SDGE,607.1,1,607.1,3.4,0,pacific_naturalgas,7.52,0.0,0.0,0.362,0.4,0.4,87,2.0,6,6,0,1.0,1.0,1.0,0,0,1,0,2.57,0.05306,false,wec_sdge_natural_gas_fired_combined_cycle_1.0
4,4,natural_gas_fired_combined_cycle,WEC_SDGE,30.0,1,30.0,4.3,0,pacific_naturalgas,10.05,0.0,0.0,0.55,0.4,0.4,87,2.0,6,6,0,1.0,1.0,1.0,0,0,2,0,2.57,0.05306,false,wec_sdge_natural_gas_fired_combined_cycle_2.0
5,5,natural_gas_fired_combined_cycle,WEC_SDGE,49.4,1,49.4,4.3,0,pacific_naturalgas,9.73,0.0,0.0,0.5,0.4,0.4,87,2.0,6,6,0,1.0,1.0,1.0,0,0,3,0,2.57,0.05306,false,wec_sdge_natural_gas_fired_combined_cycle_3.0
6,6,natural_gas_fired_combined_cycle,WEC_SDGE,570.0,1,570.0,3.4,0,pacific_naturalgas,7.15,0.0,0.0,0.351,0.4,0.4,87,2.0,6,6,0,1.0,1.0,1.0,0,0,4,0,2.57,0.05306,false,wec_sdge_natural_gas_fired_combined_cycle_4.0
7,7,natural_gas_fired_combined_cycle,WEC_SDGE,336.0,3,112.0,10.8,0,pacific_naturalgas,10.03,0.0,0.0,0.446,3.78,3.78,113,3.5,1,1,0,1.0,1.0,1.0,0,0,5,0,2.57,0.05306,false,wec_sdge_natural_gas_fired_combined_cycle_5.0
8,8,natural_gas_fired_combined_cycle,WEC_SDGE,336.0,3,112.0,10.8,0,pacific_naturalgas,10.1,0.0,0.0,0.446,3.78,3.78,113,3.5,1,1,0,1.0,1.0,1.0,0,0,6,0,2.57,0.05306,false,wec_sdge_natural_gas_fired_combined_cycle_6.0
9,9,natural_gas_fired_combustion_turbine,WEC_SDGE,91.6,2,45.8,10.8,0,pacific_naturalgas,10.69,0.0,0.0,0.8,3.78,3.78,113,3.5,1,1,0,1.0,1.0,1.0,0,0,1,0,2.57,0.05306,false,wec_sdge_natural_gas_fired_combustion_turbine_1.0


In [70]:
solution = unit_commitment_multi_time(gen_df, loads_multi, gen_variable_multi);

solution.solution
solution.cost

LoadError: The provided `optimizer_constructor` is invalid. It must be callable with zero arguments. For example, "Ipopt.Optimizer" or "() -> ECOS.Optimizer()". It should not be an instantiated optimizer like "Ipopt.Optimizer()" or "ECOS.Optimizer()". (Note the difference in parentheses!)

In [25]:
CSV.write("solution.csv", solution.solution)
marginals = marginals

solution. dual.(RampUp)

LoadError: UndefVarError: solution not defined

In [ ]:
sol_gen = innerjoin(solution.solution, 
                    gen_df[!, [:r_id, :resource]], 
                    on = :r_id)
sol_gen = combine(groupby(sol_gen, [:resource, :hour]), 
            :gen => sum)
sol_gen_btm = sol_gen
sol_gen_btm[sol_gen_btm.resource .== "solar_photovoltaic", :resource] .= "_solar_photovoltaic"
sol_gen_btm[sol_gen_btm.resource .== "onshore_wind_turbine", :resource] .= "_onshore_wind_turbine"
sol_gen_btm[sol_gen_btm.resource .== "small_hydroelectric", :resource] .= "_small_hydroelectric"


In [ ]:
btm = DataFrame(resource = repeat(["_solar_photovoltaic_btm"]; outer=length(T_period)), 
    hour = T_period,
    gen_sum = gen_variable_multi[gen_variable_multi.gen_full .== "wec_sdge_solar_photovoltaic_1.0",:cf] * 600)
append!(sol_gen_btm, btm)